# Regression between vertical motion and longwave/shortwave radiation

## Import package

In [1]:
import h5py
import numpy as np
import xarray as xr

from matplotlib import pyplot as plt

## Load data

In [2]:
# Load CloudSat radiation
CLOUDSAT_PATH = "/work/b11209013/2025_Research/CloudSat/CloudSat_sub/"

## load qlw
with xr.open_dataset(f"{CLOUDSAT_PATH}qlw.nc", chunks={}) as f:
    coords = f.coords

    qlw = f["qlw"].values

# Load ERA5 w
ERA5_PATH = "/work/b11209013/2024_Research/ERA5/w/w_Itp_sub.nc"

with xr.open_dataset(ERA5_PATH, chunks={}) as f:

    w = f["w"].values

nt, nz, ny, nx = w.shape

## Calculate linear response function between ERA5 and CloudSat

In [3]:
# reshape dataset
qlw_reshape = qlw.transpose(1, 0, 2, 3).reshape(nz, -1)
w_reshape = w.transpose(1, 0, 2, 3).reshape(nz, -1)

# filter out nan values
valid_idx = ~np.any(np.isnan(qlw_reshape), axis=0) & ~np.any(np.isnan(w_reshape), axis=0)
qlw_reshape = qlw_reshape[:, valid_idx]
w_reshape = w_reshape[:, valid_idx]

# remove mean
qlw_reshape -= qlw_reshape.mean(axis=1, keepdims=True)
w_reshape -= w_reshape.mean(axis=1, keepdims=True)

del qlw, w

# apply PCA on w
w_U, w_S, w_Vt = np.linalg.svd(w_reshape.T, full_matrices=False)

w_eofs = w_Vt.T                  # shape: (nz, n_modes)
w_pcs  = (w_U @ np.diag(w_S)).T  # shape: (n_modes, n_samples)

# # project qlw and qsw onto w EOFs
qlw_pcs =  np.linalg.inv(w_eofs.T @ w_eofs) @ w_eofs.T @ qlw_reshape  # shape: (n_modes, n_samples)

print("qlw_pcs shape:", qlw_pcs.shape)

# apply normal equation for regression coefficients
PC_reg = qlw_pcs @ w_pcs.T @ np.linalg.inv(w_pcs @ w_pcs.T)

qlw_pcs shape: (37, 2342031)


## Save File

In [4]:
with h5py.File("/home/b11209013/2025_Research/Obs/Files/ERA5/LRF_LW_w.h5", "w") as f:
    f.create_dataset("LRF", data=PC_reg)
    f.create_dataset("EOF", data=w_eofs)